In [1]:
%load_ext line_profiler
from mcts import MCTS, Go_MCTS
from bokeNet import PolicyNet, ValueNet, policy_dist, features, value
import torch

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pi = PolicyNet()
checkpt = torch.load("C:\\Users\\kyles\\Documents\\GitHub\\BokeGo\\python\\v0.2\\policy_2020-11-12_2.pt", map_location = device)
pi.load_state_dict(checkpt["model_state_dict"])
pi.eval()
val = ValueNet()
checkpt = torch.load("C:\\Users\\kyles\\Documents\\GitHub\\BokeGo\\python\\v0.2\\value_2020-11-13_6.pt", map_location = device)
val.load_state_dict(checkpt["model_state_dict"])
val.eval()
tree = MCTS(value_net=val, policy_net=pi)
board = Go_MCTS()

In [3]:
from torch.nn import MSELoss
import torch
v = ValueNet()
t = torch.randn(1,27,9,9)
loss = MSELoss()
loss(v(t), torch.Tensor(1))

C:\Users\kyles\Anaconda3\envs\finance\lib\site-packages\torch\nn\modules\loss.py:446: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(4.7173e-06, grad_fn=<MseLossBackward>)

In [4]:
%lprun -f features features(board)

Timer unit: 1e-07 s

Total time: 0.0087317 s
File: C:\Users\kyles\Documents\GitHub\BokeGo\python\bokeNet.py
Function: features at line 116

Line #      Hits         Time  Per Hit   % Time  Line Contents
   116                                           def features(game: go.Game):
   117                                               ''' go.Game --> (27,9,9) torch.Tensor
   118                                                   Compute the input features from the board state
   119                                                   
   120                                                   9x9 layer index: feature
   121                                                   ------------------------
   122                                                   0: player stones        
   123                                                       1 if coord has player's stones, else 0
   124                                                   1: opponent stones      
   125                               

In [5]:
%lprun -f tree.do_rollout tree.do_rollout(board, 100)

Timer unit: 1e-07 s

Total time: 92.8527 s
File: C:\Users\kyles\Documents\GitHub\BokeGo\python\mcts.py
Function: do_rollout at line 52

Line #      Hits         Time  Per Hit   % Time  Line Contents
    52                                               def do_rollout(self, node, n = 1):
    53                                                   "Train for n iterations"
    54       101       1215.0     12.0      0.0          for _ in range(n):
    55                                                       # Get path to leaf of current search tree
    56       100    3598828.0  35988.3      0.4              path = self._descend(node)
    57       100       1976.0     19.8      0.0              leaf = path[-1]
    58       100        957.0      9.6      0.0              if leaf.features is None:
    59        13     765262.0  58866.3      0.1                  leaf.set_features()
    60       100       1313.0     13.1      0.0              if not leaf.value:
    61        13     748636.0  5758

In [6]:
%lprun -f tree._simulate tree._simulate(board)

Timer unit: 1e-07 s

Total time: 0.893051 s
File: C:\Users\kyles\Documents\GitHub\BokeGo\python\mcts.py
Function: _simulate at line 88

Line #      Hits         Time  Per Hit   % Time  Line Contents
    88                                               def _simulate(self, node):
    89                                                   "Returns the reward for a random simulation (to completion) of `node`"
    90         1         28.0     28.0      0.0          invert_reward = not node.color
    91                                                   while True:
    92        72       1824.0     25.3      0.0              if node.terminal:
    93         1       3963.0   3963.0      0.0                  reward = node.reward()
    94         1         20.0     20.0      0.0                  reward = invert_reward^reward
    95         1          3.0      3.0      0.0                  return reward
    96        71    8924676.0 125699.7     99.9              node = node.find_random_child(self

In [7]:
%timeit tree._simulate(board)

556 ms ± 27.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%lprun -f board.find_random_child board.find_random_child(pi)

Timer unit: 1e-07 s

Total time: 0.0004061 s
File: C:\Users\kyles\Documents\GitHub\BokeGo\python\mcts.py
Function: find_random_child at line 169

Line #      Hits         Time  Per Hit   % Time  Line Contents
   169                                               def find_random_child(self, policy: PolicyNet):
   170                                                   '''Draws legal move from distribution given by policy. If no
   171                                                   policy is given, a legal move is drawn uniformly.
   172                                                   Returns a copy of the board (Go_MCTS object) after the move has
   173                                                   been played.'''
   174         1         71.0     71.0      1.7          if self.terminal:
   175                                                       return self # Game is over; no moves can be made
   176         1       3990.0   3990.0     98.3          return self.make_move(self.ge

In [9]:
%lprun -f tree._descend tree._descend(board)

Timer unit: 1e-07 s

Total time: 0.0027167 s
File: C:\Users\kyles\Documents\GitHub\BokeGo\python\mcts.py
Function: _descend at line 66

Line #      Hits         Time  Per Hit   % Time  Line Contents
    66                                               def _descend(self, node):
    67                                                   "Return a path from root down to leaf via PUCT selection"
    68                                                   # Start at root (current position)
    69         1         19.0     19.0      0.1          path = [node]
    70                                                   while True:
    71                                                       # Is node a leaf?
    72         2        169.0     84.5      0.6              if node not in self.children or not self.children[node]:
    73                                                           # Heuristic: if node is "promising" (i.e. large # of visits),
    74                                             

In [10]:
%timeit tree._descend(board)

795 µs ± 37.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
%lprun -f tree._puct_select tree._puct_select(board)

Timer unit: 1e-07 s

Total time: 0.0024685 s
File: C:\Users\kyles\Documents\GitHub\BokeGo\python\mcts.py
Function: _puct_select at line 106

Line #      Hits         Time  Per Hit   % Time  Line Contents
   106                                               def _puct_select(self, node):
   107                                                   "Select a child of node with PUCT"
   108                                           
   109                                                   # Predictor + UCT (PUCT) variant used in AlphaGo
   110         1       2357.0   2357.0      9.5          total_visits = sum(self.N[n] for n in self.children[node])
   111                                                   # First visit selects policy's top choice
   112         1         19.0     19.0      0.1          if total_visits == 0:
   113                                                       total_visits = 1
   114         1         15.0     15.0      0.1          if not node.dist:
   115            

In [12]:
%timeit tree._puct_select(board)

775 µs ± 26.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
%timeit board.make_move(board.get_move(pi))

53.1 µs ± 1.65 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [14]:
%lprun -f policy_dist policy_dist(pi, board, fts=board.features)

Timer unit: 1e-07 s

Total time: 0.0073043 s
File: C:\Users\kyles\Documents\GitHub\BokeGo\python\bokeNet.py
Function: policy_dist at line 190

Line #      Hits         Time  Per Hit   % Time  Line Contents
   190                                           def policy_dist(policy: PolicyNet,
   191                                                           game: go.Game,
   192                                                           device = torch.device("cpu"),
   193                                                           fts: torch.Tensor=None):
   194                                               '''Return torch.distribution.Categorial distribution over coordinates'''
   195         1         13.0     13.0      0.0      if fts is None:
   196                                                   fts = features(game)
   197         1       1820.0   1820.0      2.5      fts = fts.unsqueeze(0).to(device)
   198         1      70466.0  70466.0     96.5      probs = SOFT(policy(fts)).squeez

In [15]:
%timeit policy_dist(pi, board, fts=board.features)

4.69 ms ± 297 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
%lprun -f value value(val, board, fts=board.features)

Timer unit: 1e-07 s

Total time: 0.0061317 s
File: C:\Users\kyles\Documents\GitHub\BokeGo\python\bokeNet.py
Function: value at line 202

Line #      Hits         Time  Per Hit   % Time  Line Contents
   202                                           def value(v: ValueNet,
   203                                                     game: go.Game,
   204                                                     device = torch.device("cpu"),
   205                                                     fts: torch.Tensor=None):
   206         1         19.0     19.0      0.0      if fts is None:
   207                                                   fts = features(game)
   208         1        919.0    919.0      1.5      fts = fts.unsqueeze(0).to(device)
   209         1      60379.0  60379.0     98.5      return v(fts).item()

In [17]:
%timeit value(val, board, fts=board.features)

4.7 ms ± 276 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
%timeit board.get_move(pi)

35.6 µs ± 1.86 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [19]:
%lprun -f board.get_move board.get_move(pi)

Timer unit: 1e-07 s

Total time: 0.0007121 s
File: C:\Users\kyles\Documents\GitHub\BokeGo\python\mcts.py
Function: get_move at line 198

Line #      Hits         Time  Per Hit   % Time  Line Contents
   198                                               def get_move(self, policy: PolicyNet):
   199         1       6377.0   6377.0     89.6          move = self.dist_sample(policy)
   200         1        737.0    737.0     10.3          while not self.is_legal(move):
   201                                                       move = self.dist_sample(policy) 
   202         1          7.0      7.0      0.1          return move

In [21]:
%timeit board.dist_sample(pi)

24.9 µs ± 1.71 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [23]:
%lprun -f board.dist_sample board.dist_sample(pi)

Timer unit: 1e-07 s

Total time: 0.0008179 s
File: C:\Users\kyles\Documents\GitHub\BokeGo\python\mcts.py
Function: dist_sample at line 215

Line #      Hits         Time  Per Hit   % Time  Line Contents
   215                                               def dist_sample(self, policy: PolicyNet):
   216                                                   '''Sample a move from the policy distribution'''
   217         1         19.0     19.0      0.2          if not self.dist:
   218                                                       self.set_dist(policy)
   219         1       8160.0   8160.0     99.8          return self.dist.sample().item()

In [24]:
%timeit board.dist.sample().item()

25.2 µs ± 3.09 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [25]:
%lprun -f board.dist.sample board.dist.sample()

Timer unit: 1e-07 s

Total time: 0.0006583 s
File: C:\Users\kyles\Anaconda3\envs\finance\lib\site-packages\torch\distributions\categorical.py
Function: sample at line 103

Line #      Hits         Time  Per Hit   % Time  Line Contents
   103                                               def sample(self, sample_shape=torch.Size()):
   104         1         50.0     50.0      0.8          if not isinstance(sample_shape, torch.Size):
   105                                                       sample_shape = torch.Size(sample_shape)
   106         1       3856.0   3856.0     58.6          probs_2d = self.probs.reshape(-1, self._num_events)
   107         1       1068.0   1068.0     16.2          samples_2d = torch.multinomial(probs_2d, sample_shape.numel(), True).T
   108         1       1609.0   1609.0     24.4          return samples_2d.reshape(self._extended_shape(sample_shape))

In [20]:
import numpy as np

a = np.ones((1,9,9†))
b = np.zeros((7,9,9))
b

SyntaxError: invalid character in identifier (<ipython-input-20-029973daa165>, line 3)

In [ ]:
np.vstack([a,a,a,a,a,b, b]).shape

In [ ]:
import torch
from bokePolicy import ValueNet
a = torch.zeros(1,27,9,9)
v = ValueNet()
v(a)

In [ ]:
import pandas as pd
from torch.utils.data import DataLoader
from torch.nn import MSELoss
from bokeNet import NinebyNineGames, ValueNet, PolicyNet
data = NinebyNineGames("../data/hval.csv")
dataloader = DataLoader(data, batch_size = 16, shuffle = True)


In [ ]:
ft, res = data[100]
v = ValueNet()
pi = PolicyNet()
v(ft.unsqueeze(0))

In [ ]:
v = ValueNet()

loss = MSELoss()
for i, data in enumerate(dataloader, 0):
    inputs, res = data
    break
print(res)
loss(v(inputs), res.unsqueeze(1))


In [ ]:
v(inputs).view(32)

In [ ]:
x = v.conv(inputs)
x = x.view(-1, 128*9*9)
x = v.lin1(x)
v.tanh(x).view(-1)